https://github.com/PanJianning/DCIC-2019-Credit-2th-Place/tree/master/code

In [1]:
import os

os.environ['NUM_OMP_THREADS'] = "4"

import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import lightgbm as lgb
import time
from sklearn.linear_model import HuberRegressor
import sklearn.ensemble as tree_model
from tqdm import tqdm
import datetime
pd.set_option('display.max_column',100)
warnings.filterwarnings('ignore')

%load_ext autoreload
%autoreload 2
from utils import make_dir, score, timer, kf_lgbm, kf_xgbm, kf_ctbm, kf_sklearn

In [2]:
def make_features(df):
    app_feature = [
        'online_shopping_count',
        'express_count',
        'finance_app_count',
        'video_app_count',
        'flight_count',
        'train_count',
        'tour_app_count'
    ]
    
    for f in app_feature:
        df['round_log1p_' + f] = np.round(np.log1p(df[f])).astype(int)
    
    df['ago_5month_total_fee'] = 6 * df['recent_6month_avg_use'] - df['total_account_fee']
    df['ago_5month_avg_fee'] = df['ago_5month_total_fee'] / 5
    df['total_account_fee/ago_5month_total_fee'] = (df['total_account_fee']) / (1 + df['ago_5month_total_fee'])
    df['total_account_fee-ago_5month_avg_fee'] = df['total_account_fee'] - df['ago_5month_avg_fee']
        
    def make_count_feature(df, col, fea_name):
        df['idx'] = range(len(df))
        tmp = df.groupby(col)['uid'].agg([(fea_name, 'count')]).reset_index()
        df = df.merge(tmp)
        df = df.sort_values('idx').drop('idx',axis=1).reset_index(drop=True)
        return df
        
    df = make_count_feature(df, 'top_up_amount', 'count_top_up_amount')
    df = make_count_feature(df, 'total_account_fee', 'count_total_account_fee')
    df = make_count_feature(df, 'ago_5month_total_fee', 'count_ago_5month_total_fee')
    df = make_count_feature(df, 'total_account_fee-ago_5month_avg_fee', 'count_total_account_fee-ago_5month_avg_fee')
    df = make_count_feature(df, 'recent_6month_avg_use', 'count_recent_6month_avg_use')
    df = make_count_feature(df, ['total_account_fee', 'recent_6month_avg_use'], 'count_total_account_fee_recent_6month_avg_use')
            
    arr = df['top_up_amount']
    df['is_998_discount'] = ((arr / 0.998) % 1 == 0) & (arr != 0)
    
    df['age_0_as_nan'] = np.where(df['age'] == 0, [np.nan] * len(df), df['age'])
    
    return df
    
def load_df_and_make_features():
    train_df = pd.read_csv('../../input/train_dataset.csv')
    test_df = pd.read_csv('../../input/test_dataset.csv')
    
    train_df.columns = ['uid','true_name_flag','age','uni_student_flag','blk_list_flag',
                        '4g_unhealth_flag','net_age_till_now','top_up_month_diff','top_up_amount',
                        'recent_6month_avg_use','total_account_fee','curr_month_balance',
                        'curr_overdue_flag','cost_sensitivity','connect_num','freq_shopping_flag',
                        'recent_3month_shopping_count','wanda_flag','sam_flag','movie_flag',
                        'tour_flag','sport_flag','online_shopping_count','express_count',
                        'finance_app_count','video_app_count','flight_count','train_count',
                        'tour_app_count','score']
    test_df.columns = train_df.columns[:-1]
    train_df['train'] = 1
    test_df['train'] = 0
    df = pd.concat([train_df, test_df])
    df = make_features(df)
    return df

In [3]:
feature_name1 = [
    'age',
    'net_age_till_now',
    'true_name_flag',
    'uni_student_flag',
    '4g_unhealth_flag',
    'top_up_month_diff',
    'top_up_amount',
    'recent_6month_avg_use',
    'total_account_fee',
    'curr_month_balance',
    'cost_sensitivity',
    'total_account_fee-ago_5month_avg_fee',
    'ago_5month_total_fee',
    'count_top_up_amount',
    'count_total_account_fee',
    'count_total_account_fee-ago_5month_avg_fee',
    'count_recent_6month_avg_use',
    'count_total_account_fee_recent_6month_avg_use',
    'is_998_discount',
    'connect_num',
    'recent_3month_shopping_count',
    'online_shopping_count',
    'express_count',
    'finance_app_count',
    'video_app_count',
    'flight_count',
    'train_count',
    'tour_app_count',
    'wanda_flag',
    'sam_flag',
    'movie_flag',
    'tour_flag',
    'sport_flag',
    'freq_shopping_flag',
    'blk_list_flag',
    'curr_overdue_flag'
]

feature_name2 = [
    'age_0_as_nan',
    'net_age_till_now',
    'true_name_flag',
    'uni_student_flag',
    '4g_unhealth_flag',
    'top_up_month_diff',
    'top_up_amount',
    'recent_6month_avg_use',
    'total_account_fee',
    'curr_month_balance',
    'cost_sensitivity',
    'total_account_fee-ago_5month_avg_fee',
    'ago_5month_total_fee',
    'count_top_up_amount',
    'count_total_account_fee',
    'count_total_account_fee-ago_5month_avg_fee',
    'count_recent_6month_avg_use',
    'count_total_account_fee_recent_6month_avg_use',
    'is_998_discount',
    'connect_num',
    'recent_3month_shopping_count',
    'online_shopping_count',
    'express_count',
    'finance_app_count',
    'video_app_count',
    'flight_count',
    'train_count',
    'tour_app_count',
    'wanda_flag',
    'sam_flag',
    'movie_flag',
    'tour_flag',
    'sport_flag',
    'freq_shopping_flag',
    'blk_list_flag',
    'curr_overdue_flag'
]

feature_name3 = [
    'age',
    'net_age_till_now',
    'true_name_flag',
    'uni_student_flag',
    '4g_unhealth_flag',
    'top_up_month_diff',
    'top_up_amount',
    'recent_6month_avg_use',
    'total_account_fee',
    'curr_month_balance',
    'cost_sensitivity',
    'total_account_fee-ago_5month_avg_fee',
    'ago_5month_total_fee',
    'count_top_up_amount',
    'count_total_account_fee',
    'count_total_account_fee-ago_5month_avg_fee',
    'count_recent_6month_avg_use',
    'count_total_account_fee_recent_6month_avg_use',
    'is_998_discount',
    'connect_num',
    'recent_3month_shopping_count',
    'round_log1p_online_shopping_count',
    'round_log1p_express_count',
    'round_log1p_finance_app_count',
    'round_log1p_video_app_count',
    'round_log1p_flight_count',
    'round_log1p_train_count',
    'round_log1p_tour_app_count',
    'wanda_flag',
    'sam_flag',
    'movie_flag',
    'tour_flag',
    'sport_flag',
    'freq_shopping_flag',
    'blk_list_flag',
    'curr_overdue_flag'
]

feature_name4 = [
    'age_0_as_nan',
    'net_age_till_now',
    'true_name_flag',
    'uni_student_flag',
    '4g_unhealth_flag',
    'top_up_month_diff',
    'top_up_amount',
    'recent_6month_avg_use',
    'total_account_fee',
    'curr_month_balance',
    'cost_sensitivity',
    'total_account_fee-ago_5month_avg_fee',
    'ago_5month_total_fee',
    'count_top_up_amount',
    'count_total_account_fee',
    'count_total_account_fee-ago_5month_avg_fee',
    'count_recent_6month_avg_use',
    'count_total_account_fee_recent_6month_avg_use',
    'is_998_discount',
    'connect_num',
    'recent_3month_shopping_count',
    'round_log1p_online_shopping_count',
    'round_log1p_express_count',
    'round_log1p_finance_app_count',
    'round_log1p_video_app_count',
    'round_log1p_flight_count',
    'round_log1p_train_count',
    'round_log1p_tour_app_count',
    'wanda_flag',
    'sam_flag',
    'movie_flag',
    'tour_flag',
    'sport_flag',
    'freq_shopping_flag',
    'blk_list_flag',
    'curr_overdue_flag'
]

feature_name5 = [
    'age',
    'net_age_till_now',
    'true_name_flag',
    'uni_student_flag',
    '4g_unhealth_flag',
    'top_up_month_diff',
    'top_up_amount',
    'recent_6month_avg_use',
    'total_account_fee',
    'curr_month_balance',
    'cost_sensitivity',
    'total_account_fee-ago_5month_avg_fee',
    'connect_num',
    'recent_3month_shopping_count',
    'online_shopping_count',
    'express_count',
    'finance_app_count',
    'video_app_count',
    'flight_count',
    'train_count',
    'tour_app_count',
    'wanda_flag',
    'sam_flag',
    'movie_flag',
    'tour_flag',
    'sport_flag',
    'freq_shopping_flag',
    'blk_list_flag',
    'curr_overdue_flag'
]

feature_name6 = [
    'age_0_as_nan',
    'net_age_till_now',
    'true_name_flag',
    'uni_student_flag',
    '4g_unhealth_flag',
    'top_up_month_diff',
    'top_up_amount',
    'recent_6month_avg_use',
    'total_account_fee',
    'curr_month_balance',
    'cost_sensitivity',
    'total_account_fee-ago_5month_avg_fee',
    'connect_num',
    'recent_3month_shopping_count',
    'online_shopping_count',
    'express_count',
    'finance_app_count',
    'video_app_count',
    'flight_count',
    'train_count',
    'tour_app_count',
    'wanda_flag',
    'sam_flag',
    'movie_flag',
    'tour_flag',
    'sport_flag',
    'freq_shopping_flag',
    'blk_list_flag',
    'curr_overdue_flag'
]

In [4]:
df = load_df_and_make_features()
train_df = df[df['train'] == 1]
test_df = df[df['train'] != 1]

In [5]:
output_dir = 'stacking_files/'

In [6]:
x, y = train_df[feature_name1], train_df['score'].values
x_test = test_df[feature_name1]

model = kf_lgbm(x=x,y=y,x_test=x_test,learning_rate=0.01, 
                stratify=True,
                min_split_gain=1,
                categorical_feature=['cost_sensitivity'],
                boosting_type='gbdt',
                early_stopping_rounds=80,
                fair_c=25, 
                huber_delta=2,
                max_cat_to_onehot=4,
                objective="mae_fair",
                eval_metric="mae",
                subsample_freq=2,
                min_child_samples=20,
                num_leaves=31,
                bagging_fraction=0.8,
                feature_fraction=0.5,
                max_depth=5,
                output_dir=output_dir,
                name='gotcha_lgb1',
                n_estimators=8000)


================================================== Fold 1 ==================================================
Training until validation scores don't improve for 80 rounds
[500]	train's l1: 14.4725	train's mae: 14.4725	test's l1: 14.8903	test's mae: 14.8903
[1000]	train's l1: 14.0933	train's mae: 14.0933	test's l1: 14.7451	test's mae: 14.7451
[1500]	train's l1: 13.8385	train's mae: 13.8385	test's l1: 14.6922	test's mae: 14.6922
[2000]	train's l1: 13.6193	train's mae: 13.6193	test's l1: 14.6579	test's mae: 14.6579
Early stopping, best iteration is:
[2330]	train's l1: 13.4874	train's mae: 13.4874	test's l1: 14.6461	test's mae: 14.6461

================================================== Fold 2 ==================================================
Training until validation scores don't improve for 80 rounds
[500]	train's l1: 14.5281	train's mae: 14.5281	test's l1: 14.4704	test's mae: 14.4704
[1000]	train's l1: 14.1461	train's mae: 14.1461	test's l1: 14.3129	test's mae: 14.3129
[1500]	train's l

In [7]:
x, y = train_df[feature_name2], train_df['score'].values
x_test = test_df[feature_name2]

model = kf_lgbm(x=x,y=y,x_test=x_test,learning_rate=0.01, 
                stratify=True,
                min_split_gain=1,
                categorical_feature=['cost_sensitivity'],
                boosting_type='gbdt',
                early_stopping_rounds=80,
                fair_c=23, 
                huber_delta=2,
                max_cat_to_onehot=4,
                objective="fair_huber",
                eval_metric="mae",
                subsample_freq=2,
                min_child_samples=20,
                num_leaves=31,
                bagging_fraction=0.8,
                feature_fraction=0.5,
                max_depth=5,
                output_dir=output_dir,
                name='gotcha_lgb2',
                n_estimators=8000)


================================================== Fold 1 ==================================================
Training until validation scores don't improve for 80 rounds
[500]	train's l1: 14.4934	train's mae: 14.4934	test's l1: 14.9063	test's mae: 14.9063
[1000]	train's l1: 14.0941	train's mae: 14.0941	test's l1: 14.7418	test's mae: 14.7418
[1500]	train's l1: 13.8433	train's mae: 13.8433	test's l1: 14.6956	test's mae: 14.6956
[2000]	train's l1: 13.6329	train's mae: 13.6329	test's l1: 14.6673	test's mae: 14.6673
[2500]	train's l1: 13.4412	train's mae: 13.4412	test's l1: 14.6534	test's mae: 14.6534
[3000]	train's l1: 13.27	train's mae: 13.27	test's l1: 14.6433	test's mae: 14.6433
[3500]	train's l1: 13.109	train's mae: 13.109	test's l1: 14.634	test's mae: 14.634
Early stopping, best iteration is:
[3457]	train's l1: 13.1229	train's mae: 13.1229	test's l1: 14.6326	test's mae: 14.6326

================================================== Fold 2 ================================================

In [8]:
x, y = train_df[feature_name3], train_df['score'].values
x_test = test_df[feature_name3]

model = kf_lgbm(x=x,y=y,x_test=x_test,learning_rate=0.01, 
                stratify=True, 
                min_split_gain=1,
                categorical_feature=['cost_sensitivity'],
                boosting_type='gbdt',
                early_stopping_rounds=80,
                fair_c=25, 
                huber_delta=2,
                max_cat_to_onehot=4,
                objective="mae_fair",
                eval_metric="mae",
                subsample_freq=2,
                min_child_samples=20,
                num_leaves=31,
                bagging_fraction=0.8,
                feature_fraction=0.5,
                max_depth=5,
                output_dir=output_dir,
                name='gotcha_lgb3',
                n_estimators=8000)


================================================== Fold 1 ==================================================
Training until validation scores don't improve for 80 rounds
[500]	train's l1: 14.4936	train's mae: 14.4936	test's l1: 14.9001	test's mae: 14.9001
[1000]	train's l1: 14.1261	train's mae: 14.1261	test's l1: 14.7475	test's mae: 14.7475
[1500]	train's l1: 13.878	train's mae: 13.878	test's l1: 14.6923	test's mae: 14.6923
[2000]	train's l1: 13.6679	train's mae: 13.6679	test's l1: 14.6573	test's mae: 14.6573
Early stopping, best iteration is:
[2313]	train's l1: 13.549	train's mae: 13.549	test's l1: 14.6482	test's mae: 14.6482

================================================== Fold 2 ==================================================
Training until validation scores don't improve for 80 rounds
[500]	train's l1: 14.544	train's mae: 14.544	test's l1: 14.4744	test's mae: 14.4744
[1000]	train's l1: 14.1713	train's mae: 14.1713	test's l1: 14.3281	test's mae: 14.3281
[1500]	train's l1: 13.

In [9]:
x, y = train_df[feature_name4], train_df['score'].values
x_test = test_df[feature_name4]

model = kf_lgbm(x=x,y=y,x_test=x_test,learning_rate=0.01, 
                stratify=True,
                min_split_gain=1,
                categorical_feature=['cost_sensitivity'],
                boosting_type='gbdt',
                early_stopping_rounds=80,
                fair_c=23, 
                huber_delta=2,
                max_cat_to_onehot=4,
                objective="fair_huber",
                eval_metric="mae",
                subsample_freq=2,
                min_child_samples=20,
                num_leaves=31,
                bagging_fraction=0.8,
                feature_fraction=0.5,
                max_depth=5,
                output_dir=output_dir,
                name='gotcha_lgb4',
                n_estimators=8000)


================================================== Fold 1 ==================================================
Training until validation scores don't improve for 80 rounds
[500]	train's l1: 14.5096	train's mae: 14.5096	test's l1: 14.9051	test's mae: 14.9051
[1000]	train's l1: 14.1253	train's mae: 14.1253	test's l1: 14.7341	test's mae: 14.7341
[1500]	train's l1: 13.8841	train's mae: 13.8841	test's l1: 14.6797	test's mae: 14.6797
[2000]	train's l1: 13.6788	train's mae: 13.6788	test's l1: 14.6448	test's mae: 14.6448
[2500]	train's l1: 13.4972	train's mae: 13.4972	test's l1: 14.6336	test's mae: 14.6336
Early stopping, best iteration is:
[2874]	train's l1: 13.3763	train's mae: 13.3763	test's l1: 14.6226	test's mae: 14.6226

================================================== Fold 2 ==================================================
Training until validation scores don't improve for 80 rounds
[500]	train's l1: 14.5609	train's mae: 14.5609	test's l1: 14.501	test's mae: 14.501
[1000]	train's l1:

In [10]:
x, y = train_df[feature_name6], train_df['score'].values
x_test = test_df[feature_name6]

model = kf_lgbm(x=x,y=y,x_test=x_test,learning_rate=0.01, 
                stratify=True,
                min_split_gain=1,
                categorical_feature=['cost_sensitivity'],
                boosting_type='gbdt',
                early_stopping_rounds=80,
                fair_c=23, 
                huber_delta=2,
                max_cat_to_onehot=4,
                objective="fair_huber",
                eval_metric="mae",
                subsample_freq=2,
                min_child_samples=20,
                num_leaves=31,
                bagging_fraction=0.8,
                feature_fraction=0.5,
                max_depth=5,
                output_dir=output_dir,
                name='gotcha_lgb5',
                n_estimators=8000)


================================================== Fold 1 ==================================================
Training until validation scores don't improve for 80 rounds
[500]	train's l1: 14.5287	train's mae: 14.5287	test's l1: 14.9158	test's mae: 14.9158
[1000]	train's l1: 14.1462	train's mae: 14.1462	test's l1: 14.7504	test's mae: 14.7504
[1500]	train's l1: 13.91	train's mae: 13.91	test's l1: 14.6932	test's mae: 14.6932
[2000]	train's l1: 13.7114	train's mae: 13.7114	test's l1: 14.6658	test's mae: 14.6658
[2500]	train's l1: 13.5365	train's mae: 13.5365	test's l1: 14.6477	test's mae: 14.6477
Early stopping, best iteration is:
[2560]	train's l1: 13.518	train's mae: 13.518	test's l1: 14.6457	test's mae: 14.6457

================================================== Fold 2 ==================================================
Training until validation scores don't improve for 80 rounds
[500]	train's l1: 14.5613	train's mae: 14.5613	test's l1: 14.5149	test's mae: 14.5149
[1000]	train's l1: 14.

In [11]:
x, y = train_df[feature_name1], train_df['score'].values
x_test = test_df[feature_name1]

def fn_transform(x):
    return np.power(1.005, x)
def fn_reverse_transform(x):
    ret = np.log(x)/np.log(1.005)
    return ret

y = fn_transform(y)

model = kf_lgbm(x=x,y=y,x_test=x_test,learning_rate=0.03, 
                fn_reverse_transform=fn_reverse_transform,
                stratify=True, 
                split_seed=8888,
                min_split_gain=1,
                categorical_feature=['cost_sensitivity'],
                boosting_type='gbdt',
                early_stopping_rounds=80,
                fair_c=25, 
                huber_delta=2,
                max_cat_to_onehot=4,
                objective="huber",
                eval_metric="mae",
                subsample_freq=2,
                min_child_samples=20,
                num_leaves=31,
                bagging_fraction=0.8,
                feature_fraction=0.5,
                max_depth=7,
                output_dir=output_dir,
                name='gotcha_lgb6',
                verbose=200,
                n_estimators=8000)


================================================== Fold 1 ==================================================
Training until validation scores don't improve for 80 rounds
[200]	train's l1: 2.01684	train's mae: 19.4369	test's l1: 2.03953	test's mae: 19.6309
[400]	train's l1: 1.61909	train's mae: 15.2966	test's l1: 1.65438	test's mae: 15.6069
[600]	train's l1: 1.54196	train's mae: 14.4403	test's l1: 1.59319	test's mae: 14.9126
[800]	train's l1: 1.51407	train's mae: 14.1602	test's l1: 1.58069	test's mae: 14.7824
[1000]	train's l1: 1.49478	train's mae: 13.9772	test's l1: 1.5761	test's mae: 14.734
[1200]	train's l1: 1.48004	train's mae: 13.8396	test's l1: 1.57325	test's mae: 14.7054
[1400]	train's l1: 1.46965	train's mae: 13.7432	test's l1: 1.57157	test's mae: 14.689
[1600]	train's l1: 1.46331	train's mae: 13.6858	test's l1: 1.57097	test's mae: 14.683
[1800]	train's l1: 1.45823	train's mae: 13.6385	test's l1: 1.57035	test's mae: 14.6772
Early stopping, best iteration is:
[1872]	train's l1: 